In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from transformers import pipeline
from collections import Counter
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline

import os
os.environ["WANDB_DISABLED"] = "true" 

2025-08-09 16:10:42.639741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754755842.832336      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754755842.892341      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df_train = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv', header=None)
df_validation = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv', header=None)

df_train.columns = ['ID', 'Entity', 'Sentiment', 'Text']
df_validation.columns = ['ID', 'Entity', 'Sentiment', 'Text']

In [5]:
df_train.dropna(subset=['Text'], inplace=True)
df_validation.dropna(subset=['Text'], inplace=True)

labels = df_train['Sentiment'].unique()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

df_train['label'] = df_train['Sentiment'].map(label2id)
df_validation['label'] = df_validation['Sentiment'].map(label2id)

In [7]:
train_dataset = Dataset.from_pandas(df_train[['Text', 'label']].rename(columns={'Text': 'text'}))
validation_dataset = Dataset.from_pandas(df_validation[['Text', 'label']].rename(columns={'Text': 'text'}))

print("Data cleaned and prepared.")
print(f"Number of labels: {len(labels)}")
print("Label to ID mapping:", label2id)

Data cleaned and prepared.
Number of labels: 4
Label to ID mapping: {'Positive': 0, 'Neutral': 1, 'Negative': 2, 'Irrelevant': 3}


In [8]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

print("Datasets tokenized successfully!")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/73996 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Datasets tokenized successfully!


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label
)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    report_to="none"
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    compute_metrics=compute_metrics,
)

print(" Starting model fine-tuning...")
trainer.train()

print("Evaluating final model...")
final_metrics = trainer.evaluate()
print(final_metrics)

print(" Model fine-tuning complete!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 Starting model fine-tuning...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.969800
1000,0.633200
1500,0.332100
2000,0.231000
2500,0.165300
3000,0.112600
3500,0.100400
4000,0.068900
4500,0.065700


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

Evaluating final model...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.17015519738197327, 'eval_accuracy': 0.961, 'eval_f1': 0.9610505430132918, 'eval_runtime': 4.5361, 'eval_samples_per_second': 220.452, 'eval_steps_per_second': 3.527, 'epoch': 4.0}
 Model fine-tuning complete!


In [14]:
sentiment_pipeline = pipeline("sentiment-analysis", model=trainer.model, tokenizer=tokenizer)

new_texts = [
    "Nvidia's new graphics card is amazing!",
    "I'm so angry about the new update for that game, it's unplayable.",
    "This product is just okay, not great but not bad either.",
    "Why is this article about politics in my gaming feed?"
]

predictions = sentiment_pipeline(new_texts)

print("\n--- Predictions on New Sentences ---")
for text, result in zip(new_texts, predictions):
    print(f"\nText: '{text}'")
    print(f"Sentiment: {result['label']} (Score: {result['score']:.4f})")

Device set to use cuda:0



--- Predictions on New Sentences ---

Text: 'Nvidia's new graphics card is amazing!'
Sentiment: Positive (Score: 0.9991)

Text: 'I'm so angry about the new update for that game, it's unplayable.'
Sentiment: Negative (Score: 0.9999)

Text: 'This product is just okay, not great but not bad either.'
Sentiment: Irrelevant (Score: 0.9617)

Text: 'Why is this article about politics in my gaming feed?'
Sentiment: Negative (Score: 0.9974)


In [15]:
output_save_dir = "twitter_sentiment_model_final"

trainer.save_model(output_save_dir)

tokenizer.save_pretrained(output_save_dir)

print(f"Model and tokenizer have been saved to the '{output_save_dir}' directory.")

Model and tokenizer have been saved to the 'twitter_sentiment_model_final' directory.
